In [8]:
%%writefile task1.cu
#include <iostream>
#include <cuda_runtime.h>

using namespace std;

// Структура параллельного стека
struct Stack {
    int* data;
    int top;
    int capacity;

    __device__ void init(int* buffer, int size) {
        data = buffer;
        top = -1;
        capacity = size;
    }

    __device__ bool push(int value) {
        int pos = atomicAdd(&top, 1);
        if (pos < capacity) {
            data[pos] = value;
            return true;
        }
        atomicSub(&top, 1);
        return false;
    }

    __device__ bool pop(int* value) {
        int pos = atomicSub(&top, 1);
        if (pos >= 0) {
            *value = data[pos];
            return true;
        }
        atomicAdd(&top, 1);
        return false;
    }
};

// Ядро инициализации стека
__global__ void initKernel(Stack* stack, int* buffer, int size) {
    stack->init(buffer, size);
}

// Ядро параллельного добавления элементов
__global__ void pushKernel(Stack* stack, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        stack->push(idx);
    }
}

// Ядро параллельного извлечения элементов
__global__ void popKernel(Stack* stack, int* output, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        int value;
        if (stack->pop(&value)) {
            output[idx] = value;
        } else {
            output[idx] = -1;
        }
    }
}

int main() {
    const int STACK_SIZE = 128;
    const int NUM_THREADS = 128;

    // Выделение памяти под стек
    int* d_buffer;
    cudaMalloc(&d_buffer, STACK_SIZE * sizeof(int));

    Stack* d_stack;
    cudaMalloc(&d_stack, sizeof(Stack));

    // Инициализация стека
    initKernel<<<1, 1>>>(d_stack, d_buffer, STACK_SIZE);
    cudaDeviceSynchronize();

    // Параллельный push
    pushKernel<<<1, NUM_THREADS>>>(d_stack, NUM_THREADS);
    cudaDeviceSynchronize();

    // Буфер для pop
    int* d_output;
    cudaMalloc(&d_output, NUM_THREADS * sizeof(int));

    // Параллельный pop
    popKernel<<<1, NUM_THREADS>>>(d_stack, d_output, NUM_THREADS);
    cudaDeviceSynchronize();

    // Копирование результатов на CPU
    int h_output[NUM_THREADS];
    cudaMemcpy(h_output, d_output,
               NUM_THREADS * sizeof(int),
               cudaMemcpyDeviceToHost);

    // Проверка корректности
    cout << "Первые 10 извлечённых элементов:\n";
    for (int i = 0; i < 10; i++) {
        cout << h_output[i] << " ";
    }
    cout << endl;

    // Освобождение памяти
    cudaFree(d_buffer);
    cudaFree(d_stack);
    cudaFree(d_output);

    return 0;
}


Overwriting task1.cu


In [9]:
!nvcc task1.cu -o task1
!./task1

Первые 10 извлечённых элементов:
0 0 0 0 0 0 0 0 0 0 


In [10]:
%%writefile task2.cu
#include <iostream>
#include <cuda_runtime.h>

using namespace std;

// Структура параллельной очереди
struct Queue {
    int* data;
    int head;
    int tail;
    int capacity;

    // Инициализация очереди
    __device__ void init(int* buffer, int size) {
        data = buffer;
        head = 0;
        tail = 0;
        capacity = size;
    }

    // Добавление элемента в очередь
    __device__ bool enqueue(int value) {
        int pos = atomicAdd(&tail, 1);
        if (pos < capacity) {
            data[pos] = value;
            return true;
        }
        atomicSub(&tail, 1);
        return false;
    }

    // Извлечение элемента из очереди
    __device__ bool dequeue(int* value) {
        int pos = atomicAdd(&head, 1);
        if (pos < tail) {
            *value = data[pos];
            return true;
        }
        atomicSub(&head, 1);
        return false;
    }
};

// Ядро инициализации очереди
__global__ void initKernel(Queue* queue, int* buffer, int size) {
    queue->init(buffer, size);
}

// Параллельное добавление элементов
__global__ void enqueueKernel(Queue* queue, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        queue->enqueue(idx);
    }
}

// Параллельное извлечение элементов
__global__ void dequeueKernel(Queue* queue, int* output, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        int value;
        if (queue->dequeue(&value)) {
            output[idx] = value;
        } else {
            output[idx] = -1;
        }
    }
}

int main() {
    const int QUEUE_SIZE = 128;
    const int NUM_THREADS = 128;

    // Выделение памяти под данные очереди
    int* d_buffer;
    cudaMalloc(&d_buffer, QUEUE_SIZE * sizeof(int));

    // Выделение памяти под структуру очереди
    Queue* d_queue;
    cudaMalloc(&d_queue, sizeof(Queue));

    // Инициализация очереди
    initKernel<<<1, 1>>>(d_queue, d_buffer, QUEUE_SIZE);
    cudaDeviceSynchronize();

    // Параллельное добавление элементов
    enqueueKernel<<<1, NUM_THREADS>>>(d_queue, NUM_THREADS);
    cudaDeviceSynchronize();

    // Буфер для извлечённых элементов
    int* d_output;
    cudaMalloc(&d_output, NUM_THREADS * sizeof(int));

    // Параллельное извлечение элементов
    dequeueKernel<<<1, NUM_THREADS>>>(d_queue, d_output, NUM_THREADS);
    cudaDeviceSynchronize();

    // Копирование результатов на CPU
    int h_output[NUM_THREADS];
    cudaMemcpy(h_output, d_output,
               NUM_THREADS * sizeof(int),
               cudaMemcpyDeviceToHost);

    // Контрольный вывод
    cout << "Первые 10 элементов, извлечённых из очереди:\n";
    for (int i = 0; i < 10; i++) {
        cout << h_output[i] << " ";
    }
    cout << endl;

    // Освобождение памяти
    cudaFree(d_buffer);
    cudaFree(d_queue);
    cudaFree(d_output);

    return 0;
}


Writing task2.cu


In [11]:
!nvcc task2.cu -o task2
!./task2

Первые 10 элементов, извлечённых из очереди:
0 0 0 0 0 0 0 0 0 0 
